<a href="https://colab.research.google.com/github/konevenkatesh/Ontology_PM/blob/main/iprok_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install owlready2
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 30.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.47-cp311-cp311-linux_x86_64.whl size=24521242 sha256=6da10e21428153cbd5a4cacc6baed9d705bb3367bde98fe9e7f9085815783031
  Stored in directory: /root/.cache/pip/wheels/25/9a/a3/fb1ac6339caa859c8bb18d685736168b0b51d851af13d81d52
Successfully built owlready2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 8.5 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
from owlready2 import *
from datetime import date, datetime, timedelta
from rdflib import Graph

In [ ]:
set_log_level(9)

In [ ]:
# Define the IproK Ontology IRI and get/create the ontology
integrated_iri = "https://w3id.org/IproK/integrated#"
integrated_iprok = get_ontology(integrated_iri)

* Owlready2 * Creating new ontology integrated <https://w3id.org/IproK/integrated#>.
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/integrated http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology


In [ ]:
# Improt base IproK ontology
iprok = get_ontology("iprok.owl").load()
schedule_iprok = get_ontology("iprok_schedule.owl").load()
resource_iprok = get_ontology("iprok_resource.owl").load()
cost_iprok = get_ontology("iprok_cost.owl").load()
integrated_iprok.imported_ontologies.append(iprok)
integrated_iprok.imported_ontologies.append(schedule_iprok)
integrated_iprok.imported_ontologies.append(resource_iprok)
integrated_iprok.imported_ontologies.append(cost_iprok)

* Owlready2 * Reseting property iprok.hasGlobalId: new triples are now available.
* Owlready2 * Reseting property iprok.hasId: new triples are now available.
* Owlready2 * Reseting property iprok.hasName: new triples are now available.
* Owlready2 * Reseting property iprok.hasDescription: new triples are now available.
* Owlready2 * Reseting property iprok.hasGlobalId: new triples are now available.
* Owlready2 * Reseting property iprok.hasId: new triples are now available.
* Owlready2 * Reseting property iprok.hasName: new triples are now available.
* Owlready2 * Reseting property iprok.hasDescription: new triples are now available.
* Owlready2 * Reseting property iprok.hasId: new triples are now available.
* Owlready2 * Reseting property iprok.hasName: new triples are now available.


* Owlready2 * Creating new ontology iprok <iprok.owl#>.
* Owlready2 * ADD TRIPLE iprok.owl http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology
* Owlready2 *     ...loading ontology iprok from iprok.owl...
* Owlready2 *     ...4 properties found: hasGlobalId, hasId, hasName, hasDescription
* Owlready2 * Creating new ontology iprok_schedule <iprok_schedule.owl#>.
* Owlready2 * ADD TRIPLE iprok_schedule.owl http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology
* Owlready2 *     ...loading ontology iprok_schedule from iprok_schedule.owl...
* Owlready2 *     ...29 properties found: hasWorkPackage, hasWBS, hasTask, hasSubTask, hasTaskTime, hasPredecessorDependency, hasSuccessorDependency, linksToPredecessorTask, linksToSuccessorTask, hasConstraint, assignedTo, projectCategory, projectOwner, projectAddress, isCriticalTask, isMilestone, taskPriority, plannedStartDate, plannedFinishDate, actualStartDate, actualFinishDate, sched

In [ ]:
with integrated_iprok:
    # =======================================================================
    # Interrelation Module (Conceptual Links, e.g., from Module.docx Fig 6)
    # =======================================================================
    print("Defining Interrelation Properties...")

    # Key class for linking Task and Resource with allocation details
    class ResourceAllocation(iprok.IproKThing):
        """Represents the assignment of a specific quantity of a resource to a task over a period."""
        pass

    # Data properties for ResourceAllocation
    class allocatedUnits(DataProperty, FunctionalProperty):
        domain = [ResourceAllocation]
        range = [float]

    class allocationPlannedStartDate(DataProperty, FunctionalProperty):
        domain = [ResourceAllocation]
        range = [datetime]

    class allocationPlannedFinishDate(DataProperty, FunctionalProperty):
        domain = [ResourceAllocation]
        range = [datetime]

    # ... add actual/baseline dates for allocation if needed

    # Object properties for ResourceAllocation
    class allocationAssignsResource(ObjectProperty, FunctionalProperty):
        domain = [ResourceAllocation]
        range = [resource_iprok.Resource]

    class allocationForTask(ObjectProperty, FunctionalProperty):
        domain = [ResourceAllocation]
        range = [schedule_iprok.Task]

    # Linking Task to Resource via ResourceAllocation
    class taskHasResourceAllocation(ObjectProperty):
        domain = [schedule_iprok.Task]
        range = [ResourceAllocation]
        # Inverse: resourceAllocationIsForTask (defined on ResourceAllocation)
        # This replaces the direct 'taskRequiresResource' for detailed allocation

    # Linking Task to CostItem (already conceptualized in Schedule Module's Figure 3)
    class taskGeneratesCostItem(ObjectProperty): # More specific than just hasCostItem
        domain = [schedule_iprok.Task]
        range = [cost_iprok.CostItem]
        # Inverse: costItemGeneratedByTask

    # Linking Resource (via its allocation/usage) to CostItem
    class resourceAllocationIncursCostItem(ObjectProperty):
        domain = [ResourceAllocation]
        range = [cost_iprok.CostItem]
        # Inverse: costItemFromResourceAllocation


    print("IproK Ontology definition complete.")

Defining Interrelation Properties...
IproK Ontology definition complete.


* Owlready2 * ADD TRIPLE https://w3id.org/IproK/integrated#ResourceAllocation http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/integrated#ResourceAllocation http://www.w3.org/2000/01/rdf-schema#subClassOf https://w3id.org/IproK#IproKThing
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/integrated#allocatedUnits http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#DatatypeProperty
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/integrated#allocatedUnits http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#FunctionalProperty
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/integrated#allocatedUnits http://www.w3.org/2000/01/rdf-schema#domain https://w3id.org/IproK/integrated#ResourceAllocation
* Owlready2 * ADD TRIPLE https://w3id.org/IproK/integrated#allocatedUnits http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#decimal
* Owlread

In [ ]:
integrated_iprok.save(file="iprok_integrated.owl", format="rdfxml")

* Owlready2 * Saving ontology integrated to iprok_integrated.owl...


In [ ]:
integrated_iprok.destroy()